# Football in Denmark: Where are we playing?

Imports and set magics:

In [1]:
# %pip install git+https://github.com/alemartinello/dstapi #Installing the API (only need to do once)

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#Installing'


In [2]:
# %pip install pandas-datareader # Installing the data reader (only need to do once)

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})
import ipywidgets as widgets
# from matplotlib_venn import venn2
from dstapi import DstApi # install with `pip install git+https://github.com/alemartinello/dstapi`

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject


ModuleNotFoundError: No module named 'dstapi'

# Read and clean data

Creating a dictionary for the columns names:

In [5]:
columns_dict = {}
columns_dict['BLSTKOM'] = 'county'
columns_dict['AKTIVITET'] = 'activity'
columns_dict['KON'] = 'sex'
columns_dict['ALDER1'] = 'age'
columns_dict['TID'] = 'year'
columns_dict['INDHOLD'] = 'value'

#var_dict = {} # var is for variable
#var_dict['Football'] = 'football'

**Step 1:** Downloading all of the football-variables in IDRAKT01

In [6]:
idrakt_api = DstApi('IDRAKT01')  #Creating the DST API which will allow us to interact with the API server
params = idrakt_api._define_base_params(language='en') #Creating a parameter dictionary with the language set to English
variables = params['variables'] # Returns a view, that we can edit
variables[1]['values'] = ['A22'] # Choosing football as the activity (The ID for football is A22 Using code from: https://alemartinello.com/2022/02/24/dstapi/)
print(variables)

[{'code': 'BLSTKOM', 'values': ['*']}, {'code': 'AKTIVITET', 'values': ['A22']}, {'code': 'KON', 'values': ['*']}, {'code': 'ALDER1', 'values': ['*']}, {'code': 'Tid', 'values': ['*']}]


In [7]:
idrakt = idrakt_api.get_data(params=params) #Downloading the dataset

In [8]:
idrakt.head() #Looking at the dataset

,BLSTKOM,AKTIVITET,KON,ALDER1,TID,INDHOLD
0,Herning,Football,Men,"Age, total",2017,4950
1,Herning,Football,Men,0-12 years,2017,1850
2,Herning,Football,Men,13-18 years,2017,1070
3,Herning,Football,Men,19-24 years,2017,660
4,Herning,Football,Men,25-59 years,2017,1250


**Step 2:** Rename coloums using colmns_dict

In [9]:
idrakt.rename(columns=columns_dict,inplace=True)
idrakt.head()

,county,activity,sex,age,year,value
0,Herning,Football,Men,"Age, total",2017,4950
1,Herning,Football,Men,0-12 years,2017,1850
2,Herning,Football,Men,13-18 years,2017,1070
3,Herning,Football,Men,19-24 years,2017,660
4,Herning,Football,Men,25-59 years,2017,1250


**Step 3:** Only keep rows where the variable is in `Age, total` and afterwards deleting the coloumn.

In [10]:
#Only keeps rows with age = 'Age, total' and afterwards deleting the age coloumn
idrakt = idrakt[idrakt['age'] == 'Age, total']
idrakt.drop(columns=['age'],inplace=True)
idrakt.head()

,county,activity,sex,year,value
0,Herning,Football,Men,2017,4950
6,Herning,Football,"Sex, total",2017,6240
12,Herning,Football,Women,2017,1290
18,Horsens,Football,Men,2017,4580
24,Horsens,Football,"Sex, total",2017,6240


**Step 4:** Only keep rows where the variable is in a region or total

In [11]:
# Keeping rows where 'county' starts with "Landsdel" or is "All Denmark"
idrakt = idrakt[idrakt['county'].str.startswith('Province') | (idrakt['county'] == 'All Denmark')]
idrakt.head()

,county,activity,sex,year,value
180,Province Nordjylland,Football,Men,2019,30800
186,Province Nordjylland,Football,"Sex, total",2019,40140
192,Province Nordjylland,Football,Women,2019,9340
288,Province Byen København,Football,Men,2017,43840
294,Province Byen København,Football,"Sex, total",2017,49680


**Step 5:** Sort the dataset by county, year and sex

In [12]:
# Sorting the dataset by county then year and then total sex, but first we customly sort the order of Sex
# Define a custom sorting order
sort_order = ['Sex, total', 'Men', 'Women']

# Create a categorical type with the custom order
idrakt['sex'] = pd.Categorical(idrakt['sex'], categories=sort_order, ordered=True)

idrakt.sort_values(by=['county','year','sex'],inplace=True)
idrakt.reset_index(drop=True,inplace=True)
idrakt.head()

,county,activity,sex,year,value
0,All Denmark,Football,"Sex, total",2014,374400
1,All Denmark,Football,Men,2014,307430
2,All Denmark,Football,Women,2014,66970
3,All Denmark,Football,"Sex, total",2015,372640
4,All Denmark,Football,Men,2015,306540


In [13]:
# Creating a copy of the dataset where we only keep the rows with the value 'All Denmark' in the 'county' column
#idrakt_all = idrakt[idrakt['county'] == 'All Denmark'].copy()
#idrakt_all.drop(columns=['county'],inplace=True)
#idrakt_all.head()

In [14]:
#Sorting the total dataset by year
#idrakt_all.sort_values(by='year',inplace=True)
#idrakt_all.head()

## Explore each data set

Exploring the dataset with descriptive statistics

# Analysis

Plotting the time series

Create a function that can be used to create a plot for each region

Plotting the time series for each sex in each region

# Conclusion

ADD CONCISE CONLUSION.

We see that...